* 진형님 LDA 참고하여 진행
* 동현님 gensim 파일 참고하여 진행

# 1. 데이터 로드

저장된 데이터 가져오기

In [3]:
import pandas as pd
import re
import os

In [4]:
import matplotlib.pyplot as plt

# 한글 깨짐 방지
plt.rcParams['font.family'] = "Malgun Gothic"

# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

In [5]:
# 파일 불러오기 
current_dir = os.path.dirname(os.path.abspath('df_labor_real_cleaned3.csv'))
data_dir = os.path.join(current_dir, '..', 'data')
file_path = os.path.join(data_dir, 'df_labor_real_cleaned3.csv')  # 실제 파일명으로 변경

# CSV 파일 읽기
data = pd.read_csv(file_path)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   판례정보일련번호      5016 non-null   int64 
 1   사건번호          5016 non-null   object
 2   선고            5016 non-null   object
 3   판결유형          5016 non-null   object
 4   판시사항          3725 non-null   object
 5   판결요지          3054 non-null   object
 6   참조조문          3724 non-null   object
 7   참조판례          2450 non-null   object
 8   판례내용          5016 non-null   object
 9   판례내용_상단       5016 non-null   object
 10  판례내용_이유       5012 non-null   object
 11  판례내용_이유(전처리)  5016 non-null   object
 12  판례내용_제거(불용어)  5016 non-null   object
 13  판례내용_이유(불용어)  5012 non-null   object
dtypes: int64(1), object(13)
memory usage: 548.8+ KB


In [11]:
df_labor = data['판례내용_이유(불용어)'].copy()
df_labor.info()

<class 'pandas.core.series.Series'>
RangeIndex: 5016 entries, 0 to 5015
Series name: 판례내용_이유(불용어)
Non-Null Count  Dtype 
--------------  ----- 
5012 non-null   object
dtypes: object(1)
memory usage: 39.3+ KB


# 2. 데이터 전처리

## 2-1 토큰화

* 형태소 분석 및 전처리: Okt를 사용하여 한국어 문서에서 명사만을 추출하고, 불용어를 제거합니다. 또한, 한 글자 단어들은 제거하여 의미가 있는 단어들만 남깁니다.
* 단어 사전 및 코퍼스 생성: Dictionary 객체를 사용해 단어 사전을 만들고, 이를 통해 문서 코퍼스를 bag-of-words 형식으로 변환합니다.
* LDA 모델 학습: 주제 수(num_topics)를 설정한 후 LDA 모델을 학습합니다.
주제별 핵심 단어 출력: print_topics 메서드를 사용하여 각 주제의 상위 단어를 출력합니다.

In [12]:
import pandas as pd
from konlpy.tag import Okt
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from nltk.corpus import stopwords
import re

# 예시 데이터프레임 준비 (실제 데이터프레임을 여기에 할당하세요)
# 형태소 분석기 초기화
okt = Okt()

# 불용어 설정 (필요시 추가 가능)
# stop_words = set(["이다", "있다", "하다", "되다", "않다", "하다", "보다", "등"])

# # 1. 텍스트 데이터 전처리 및 토큰화
# def preprocess_text(text):
#     # 특수문자 제거 및 소문자 변환
#     text = re.sub(r'[^\w\s]', '', text)
#     tokens = okt.nouns(text)  # 명사 추출
#     return [word for word in tokens if word not in stop_words and len(word) > 1]  # 불용어 및 한 글자 단어 제거

# 전처리 수행
documents = df_labor.dropna().tolist()  # NaN 값 제거
tokenized_docs = [preprocess_text(doc) for doc in documents]

# 2. 단어 사전 및 코퍼스 생성
dictionary = Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(text) for text in tokenized_docs]

# 3. LDA 모델 학습
num_topics = 5  # 주제 개수 설정
lda_model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=42)

# 4. 주제별 핵심 단어 출력
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

Topic 0: 0.050*"지급" + 0.039*"임금" + 0.033*"근로" + 0.027*"근로자" + 0.019*"퇴직금" + 0.019*"수당" + 0.016*"퇴직" + 0.014*"계약" + 0.014*"근무" + 0.013*"근로기준법"
Topic 1: 0.021*"보험" + 0.014*"재해" + 0.012*"사고" + 0.009*"망인" + 0.008*"급여" + 0.007*"손해" + 0.007*"산업" + 0.007*"장해" + 0.005*"작업" + 0.005*"근로자"
Topic 2: 0.024*"회사" + 0.021*"노동조합" + 0.019*"징계" + 0.018*"근로자" + 0.017*"해고" + 0.011*"단체" + 0.010*"협약" + 0.009*"노동" + 0.008*"사업" + 0.008*"노조"
Topic 3: 0.026*"소득" + 0.013*"사업" + 0.013*"금액" + 0.011*"법인" + 0.009*"세법" + 0.008*"과세" + 0.008*"부과" + 0.008*"징수" + 0.006*"납부" + 0.006*"세액"
Topic 4: 0.027*"회사" + 0.012*"주식회사" + 0.009*"채권" + 0.008*"지급" + 0.008*"계약" + 0.007*"사업" + 0.005*"명의" + 0.005*"거래" + 0.005*"주식" + 0.005*"순번"


In [15]:
import gensim
print (gensim.__version__)

4.3.3


In [ ]:
from gensim.summarization import summarize

summary = summarize(df['판례내용'].str.cat(sep=' '), ratio=0.1)  # 10% 비율로 요약
print(summary)